In [98]:
import pandas as pd
import numpy as np

In [99]:
urbn = pd.read_csv('/content/urban_noise_levels.csv')
urbn.duplicated().sum()

np.int64(0)

In [100]:
urbn.head()

,id,latitude,longitude,datetime,decibel_level,hour,day_of_week,is_weekend,temperature_c,humidity_%,...,population_density,park_proximity,industrial_zone,vehicle_count,honking_events,public_event,holiday,school_zone,noise_complaints,sensor_id
0,1,40.649816,-74.069147,2023-04-01 18:50:00,78.286408,18,5,1,16.710979,43.054747,...,28384,0,0,24,1,0,0,1,1,31
1,2,40.880286,-74.076511,2023-03-18 04:12:00,68.131845,4,5,1,10.088749,29.676246,...,24206,0,0,15,2,0,0,0,0,40
2,3,40.792798,-73.746873,2023-12-03 05:43:00,58.934966,5,6,1,19.911075,47.932832,...,1894,1,0,15,2,0,0,0,0,41
3,4,40.739463,-74.075227,2023-06-20 00:44:00,69.559042,0,1,0,3.477520,88.594010,...,2265,1,0,24,4,0,0,1,2,50
4,5,40.562407,-74.064025,2023-12-28 12:06:00,60.409097,12,3,0,17.356303,35.811699,...,23807,1,0,25,2,0,0,0,0,8


In [101]:
urbn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  2000 non-null   int64  
 1   latitude            2000 non-null   float64
 2   longitude           2000 non-null   float64
 3   datetime            2000 non-null   object 
 4   decibel_level       2000 non-null   float64
 5   hour                2000 non-null   int64  
 6   day_of_week         2000 non-null   int64  
 7   is_weekend          2000 non-null   int64  
 8   temperature_c       2000 non-null   float64
 9   humidity_%          2000 non-null   float64
 10  wind_speed_kmh      2000 non-null   float64
 11  precipitation_mm    2000 non-null   float64
 12  traffic_density     2000 non-null   int64  
 13  near_airport        2000 non-null   int64  
 14  near_highway        2000 non-null   int64  
 15  near_construction   2000 non-null   int64  
 16  popula

In [102]:
urbn['datetime'] = (pd.to_datetime(urbn['datetime'])).astype('int64')//10**9 #дата переводится в формат int в виде секунд
urbn_cat = pd.get_dummies(urbn['day_of_week'], prefix='day', dtype=int) #дни недели переводятся в категориальные признаки
urbn_cat['sin_hour'] = np.sin(2 * np.pi * urbn['hour'] / 24) #вычисляются синус и косинус числа часов, чтобы не создавать
urbn_cat['cos_hour'] = np.cos(2 * np.pi * urbn['hour'] / 24) #много категориальных признаков и избежать линейной зависимости с датой

y = urbn['decibel_level']


urbn_cat

,day_0,day_1,day_2,day_3,day_4,day_5,day_6,sin_hour,cos_hour
0,0,0,0,0,0,1,0,-1.000000e+00,-1.836970e-16
1,0,0,0,0,0,1,0,8.660254e-01,5.000000e-01
2,0,0,0,0,0,0,1,9.659258e-01,2.588190e-01
3,0,1,0,0,0,0,0,0.000000e+00,1.000000e+00
4,0,0,0,1,0,0,0,1.224647e-16,-1.000000e+00
...,...,...,...,...,...,...,...,...,...
1995,0,0,0,1,0,0,0,0.000000e+00,1.000000e+00
1996,0,0,0,0,0,0,1,8.660254e-01,-5.000000e-01
1997,0,0,0,0,0,1,0,2.588190e-01,-9.659258e-01
1998,0,0,0,0,0,1,0,-8.660254e-01,-5.000000e-01


In [103]:
#удаляются ненужные столбцы и оставшиеся (числовые) признаки масштабируются
from sklearn.preprocessing import MinMaxScaler
urbn_num = urbn.drop(columns=['id','hour','day_of_week','decibel_level','sensor_id'])
scaler = MinMaxScaler()
urbn_scaled = scaler.fit_transform(urbn_num)
urbn_scaled = pd.DataFrame(urbn_scaled,columns=urbn_num.columns)
urbn_scaled

,latitude,longitude,datetime,is_weekend,temperature_c,humidity_%,wind_speed_kmh,precipitation_mm,traffic_density,near_airport,...,near_construction,population_density,park_proximity,industrial_zone,vehicle_count,honking_events,public_event,holiday,school_zone,noise_complaints
0,0.372626,0.261813,0.248507,1.0,0.477205,0.329426,0.718875,0.030017,0.75,0.0,...,0.0,0.944535,0.0,0.0,0.53125,0.1,0.0,0.0,1.0,0.2
1,0.950824,0.247079,0.208447,1.0,0.328548,0.138232,0.392194,0.000327,0.25,0.0,...,0.0,0.800331,0.0,0.0,0.25000,0.2,0.0,0.0,0.0,0.0
2,0.731336,0.906655,0.921534,1.0,0.549041,0.399139,0.905517,0.049011,0.75,0.0,...,0.0,0.030235,1.0,0.0,0.25000,0.2,0.0,0.0,0.0,0.0
3,0.597532,0.249648,0.465797,0.0,0.180138,0.980234,0.560307,0.405988,0.75,0.0,...,0.0,0.043040,1.0,0.0,0.53125,0.4,0.0,0.0,1.0,0.4
4,0.153337,0.272062,0.990812,0.0,0.491691,0.225914,0.450603,0.138846,0.50,0.0,...,1.0,0.786560,1.0,0.0,0.56250,0.2,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.656033,0.442296,0.586402,0.0,0.609924,0.819423,0.945108,0.058279,0.75,0.0,...,0.0,0.731923,0.0,0.0,0.53125,0.6,0.0,1.0,0.0,0.2
1996,0.956746,0.334541,0.307615,1.0,0.445566,0.996471,0.272989,0.129610,0.50,0.0,...,1.0,0.942360,0.0,0.0,0.25000,0.1,0.0,0.0,0.0,0.0
1997,0.065971,0.394740,0.823491,1.0,0.180651,0.329614,0.169643,0.074146,0.75,0.0,...,0.0,0.974908,0.0,0.0,0.31250,0.4,0.0,0.0,0.0,0.0
1998,0.054026,0.530170,0.209822,1.0,0.689636,0.381713,0.484231,0.056514,0.25,0.0,...,1.0,0.262934,1.0,1.0,0.40625,0.3,0.0,0.0,1.0,0.0


In [104]:
x = pd.concat([urbn_scaled,urbn_cat],axis=1)

In [108]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=382)

lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train)
print(r2_score(y_test,lin_reg.predict(x_test)),r2_score(y_train,lin_reg.predict(x_train)))
lin_ridge = Ridge(alpha=1)
lin_ridge.fit(x_train, y_train)

print(r2_score(y_test,lin_ridge.predict(x_test)),r2_score(y_train,lin_ridge.predict(x_train)))



0.006563675286629156 0.018223284044197707
0.00666136171417453 0.01822160652374427
